In [1]:
import numpy as np
import matplotlib as plt
from keras.models import Sequential, load_model
from PIL import Image
import timeit,time
import math
import pandas as pd

from skimage import measure
from scipy import ndimage as ndi
from skimage import feature

Using TensorFlow backend.


In [42]:
path = '/home/grads/n/narendra5/Desktop/Programs/LER_machine_learning/'
#path = r"C:\\Users\\narendra\\Documents\\LER\\LER_machine_learning\\"
#model = load_model(path + 'models/' + 'EDGEnet_run_epoch_4.h5')

model = load_model(path + 'models/' + 'EDGEnet2_runL1_epoch_7.h5')

In [43]:
sigmas = [0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8]
#Xis = [10, 30, 40]
Xis = [40]
#alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
alphas = [0.9]
widths = [20, 30]
#noises = [2, 3, 4, 5, 10, 20, 30, 50, 100, 200]
noises = [2]



testsize = len(sigmas)*len(Xis)*len(alphas)*len(widths)*len(noises)*2
print('Testsize: ', testsize)


df = pd.DataFrame(columns = ['noise', 'sigma', 'alpha', 'Xi', 'width', 'space', 'MSE_noise', \
                             'PSNR_noise', 'Pred_time', 'i_leftline_sigma', 'ledge_pred_sigma',\
                             'i_rightline_sigma',  'redge_pred_sigma',\
                              'lpred_error', 'rpred_error'])
                             

count = 0
for sigma in sigmas:
    for alpha in alphas:
        for Xi in Xis:
            for width in widths:
                for s in range(2):
                    for noise in noises:
                        space = math.floor(width*2**s)
                        shift = math.floor(-25 + (width + space/2 + Xi + alpha*10 + sigma*10)%16) 
                        
                        original_file = path + 'original_images/oim_' + "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '.tiff'
                        noisy_file = path + 'noisy_images/nim_' + "{0:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '_' + str(-shift) + '_' + str(noise) + '.tiff'
                        linescan_file = path + 'linescans/linescan_' + "{:.2g}".format(sigma*1e-09) + '_' + str(alpha) + '_' + "{0:.2g}".format(Xi*1e-09) + '_' + str(width) + '_' + str(space) + '.txt'
                        linescan = []
                        
                        with open(linescan_file,'r') as f:
                            for i, line in enumerate(f):
                                if i < 3000:
                                    a, b = line.split(',')
                                    linescan.append(float(b))
                        
                        linescan = linescan[:2048]
                        
                        leftline = np.array(linescan[:1024]) 
                        rightline = linescan[1024:]
                        rightline.reverse()
                        rightline = np.array(rightline)

                        leftline = leftline + shift           # add shift to linescan
                        rightline = rightline + shift         # add shift to linescan
                        
                        im = np.array(Image.open(original_file))
                        imnoisy = np.array(Image.open(noisy_file))
                        
                        im = im/256
                        imnoisy = imnoisy/256
                        imnoisy = imnoisy.reshape(1,1024,64,1)
                        im = im.reshape(1,1024,64,1)
                        
                        start = time.time()
                        linepredict = model.predict(imnoisy)
                        
                        prediction_time = time.time() - start
                        
                        linepredict = linepredict.reshape(1024,2)
                        
                        imnoisy = imnoisy.reshape(1024,64)
                        im = im.reshape(1024,64)
                        imnoisy = imnoisy.astype(float)
                        im = im.astype(float)
                        
                        mse_noisy = (256**2)*((im - imnoisy) ** 2).mean()
                        
                        psnr_noisy = measure.compare_psnr(im,imnoisy)
                        
                        
                        ledge_pred = (linepredict*64)[:,0].round()
                        redge_pred = (linepredict*64)[:,1].round()
                        
                        
                        #lpred_rmse = np.sqrt(((leftline.astype(int) - ledge_pred)**2).mean())
                        #rpred_rmse = np.sqrt(((rightline.astype(int) - redge_pred)**2).mean())
                        lline_std = leftline.astype(int).std()/2
                        rline_std = rightline.astype(int).std()/2
                        lpred_std = (ledge_pred).std()/2
                        rpred_std = (redge_pred).std()/2
                        lerror = (lline_std - lpred_std)*100/lline_std
                        rerror = (rline_std - rpred_std)*100/rline_std
                        
                        df.loc[count] = [noise, sigma, alpha, Xi, width, space, mse_noisy, \
                                         psnr_noisy, prediction_time, \
                                         lline_std, lpred_std,\
                                         rline_std, rpred_std,\
                                         lerror, rerror]
                        
                        count += 1
                        
print("Test count: ", count)

Testsize:  32
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
Test count:  32


In [44]:
print(df)

    noise  sigma  alpha    Xi  width  space    MSE_noise  PSNR_noise  \
0     2.0    0.4    0.9  40.0   20.0   20.0  6092.806595   10.316625   
1     2.0    0.4    0.9  40.0   20.0   40.0  6210.455643   10.233565   
2     2.0    0.4    0.9  40.0   30.0   30.0  6661.252426    9.929240   
3     2.0    0.4    0.9  40.0   30.0   60.0  6626.432449    9.952002   
4     2.0    0.6    0.9  40.0   20.0   20.0  6292.165115   10.176798   
5     2.0    0.6    0.9  40.0   20.0   40.0  5911.373962   10.447915   
6     2.0    0.6    0.9  40.0   30.0   30.0  6526.628448   10.017910   
7     2.0    0.6    0.9  40.0   30.0   60.0  6638.942947    9.943810   
8     2.0    0.8    0.9  40.0   20.0   20.0  6338.165070   10.145164   
9     2.0    0.8    0.9  40.0   20.0   40.0  6039.097366   10.355079   
10    2.0    0.8    0.9  40.0   30.0   30.0  6564.057327    9.993076   
11    2.0    0.8    0.9  40.0   30.0   60.0  6803.041077    9.837768   
12    2.0    1.0    0.9  40.0   20.0   20.0  6108.636230   10.30